In [ ]:
# SOURCE:  https://github.com/atreadw1492/yahoo_fin/blob/master/yahoo_fin/stock_info.py

In [ ]:
# finviz:  https://www.youtube.com/watch?v=cGL1FJKEfAk

# Other examples
# https://www.youtube.com/watch?v=Ibgk3L96kco

In [ ]:
# DOCU:   https://theautomatic.net/yahoo_fin-documentation/

## Install libraries

In [ ]:
# !pip install yahoo_fin

In [ ]:
# !pip install pandas_datareader

## Import libraries

In [1]:
# this are from my .py files
import yahoo_finance_stocks as yf
import pandas_datareader_stocks as datareader

In [59]:
import datetime as dt
import pandas as pd

## Correlation SPY AND VIX

In [ ]:
# I want to validate when VIX and SPY open negative, so SPY close positive

In [131]:
start = dt.datetime(2022, 1, 1)
end   = dt.datetime.today()  # almost today

cols = ['Open', 'Close']

df_vix = datareader.get_ticker('^VIX',start,end)[cols]

## GAP up or GAP down.
df_vix["prev_close"] = df_vix.Close.shift(1)
df_vix['chg%_premarket'] = 100*-1*df_vix[['Open','prev_close']].pct_change(axis=1)['prev_close'].round(5)
df_vix.reset_index(inplace=True)

cols_print = ['Date','prev_close', 'chg%_premarket'] + cols
df_vix[cols_print].tail(3)

,Date,prev_close,chg%_premarket,Open,Close
200,2022-10-19,30.50,1.422,30.940001,30.760000
201,2022-10-20,30.76,1.725,31.299999,29.980000
202,2022-10-21,29.98,0.761,30.209999,29.690001


In [132]:
cols = ['open', 'close', 'volume']
df_spy = yf.get_data_startdate("SPY", '01/01/2022')[cols]


## GAP up or GAP down.
df_spy["prev_close"] = df_spy.close.shift(1)
df_spy['chg%_premarket'] = 100*-1*df_spy[['open','prev_close']].pct_change(axis=1)['prev_close'].round(5)

## Cuanto movió en el dia.
df_spy['chg%_intraday'] =  100*-1*((df_spy['open'] - df_spy['close'])/df_spy['close']).round(5)

df_spy.reset_index(inplace=True)
df_spy.rename(columns={"index":"Date"},inplace=True)

## esta es una funcion nativa
#df_spy['Chg%_intraday2'] = 100*df_spy[['open','close']].pct_change(axis=1)['close'].round(5)
cols_print = ['Date','prev_close', 'chg%_premarket', 'open', 'close', 'volume','chg%_intraday']
df_spy[cols_print].tail(3)

,Date,prev_close,chg%_premarket,open,close,volume,chg%_intraday
200,2022-10-19,371.130005,-0.580,368.989990,368.500000,79746900,-0.133
201,2022-10-20,368.500000,-0.128,368.029999,365.410004,88283100,-0.717
202,2022-10-21,365.410004,-0.079,365.119995,374.290009,130897700,2.450


In [133]:
#. join both dataframes

In [124]:
def filter_df_vix_spy(df, filter1, filter2):
    return df.where(filter1 & filter2).dropna()

def count_stats_vix_spy(df, condition, filter1):
    df2 = df.where(filter1).dropna()
    df2["condition"] = condition
    
    return df2

In [125]:
df_join_vix_spy = pd.merge(
    df_vix,
    df_spy,
    how="inner",
    on="Date",
    suffixes=("_vix", "_spy")
)

df_join_vix_spy.tail(40)

In [126]:
cols=["Date","chg%_premarket_vix","chg%_premarket_spy","chg%_intraday"]
df_vix_spy = df_join_vix_spy[cols]

In [127]:
filter1 = df_vix_spy["chg%_premarket_vix"] < 1
filter2 = df_vix_spy["chg%_premarket_spy"] < 0

df_vix_spy_neg = filter_df_vix_spy(df_vix_spy,filter1,filter2)
df_vix_spy_neg.tail(5)

,Date,chg%_premarket_vix,chg%_premarket_spy,chg%_intraday
695,2022-10-05,0.988,-1.227,0.981
696,2022-10-06,0.175,-0.391,-0.648
697,2022-10-07,-0.494,-1.146,-1.703
701,2022-10-13,0.089,-2.105,4.580
707,2022-10-21,0.761,-0.079,2.450


In [128]:
filterSpy1 = df_vix_spy_neg["chg%_intraday"] < 0
filterSpy2 = df_vix_spy_neg["chg%_intraday"] > 0

df_stats1 = count_stats_vix_spy(df_vix_spy_neg, "both_neg_&_spy_close_neg", filterSpy1)
df_stats2 = count_stats_vix_spy(df_vix_spy_neg, "both_neg_&_spy_close_pos", filterSpy2)

In [129]:
data = [df_stats1, df_stats2]
df_stats = pd.concat(data, ignore_index=True, sort=False)

In [130]:
df_stats_count = df_stats.groupby(['condition'])['condition'].count()
df_stats_count

condition
both_neg_&_spy_close_neg    55
both_neg_&_spy_close_pos    60
Name: condition, dtype: int64

## Get All Stock Symbols

## Financial: Income, Balance and Cash flow

In [ ]:
yearly_json = yf.get_financials("AAPL",True,False) # yearly
quarterly_json = yf.get_financials("AAPL",False,True) # quarterly

In [ ]:
temp1 = quarterly_json["quarterly_income_statement"]
temp2 = quarterly_json["quarterly_balance_sheet"]
temp3 = quarterly_json["quarterly_cash_flow"]

In [ ]:
##add_thousand_comma_separator(temp1,"2021-12-25")

In [ ]:
temp1 = yf.convert_dataframe_reset_index(temp1)
temp11 = yf.rename_columns_dataframe(temp1)
temp11_1 = yf.convert_quarters_to_currency_format(temp11)
temp11_1

In [ ]:
temp11.dtypes